In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import math

from sklearn.model_selection import train_test_split
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.utils.data.dataset import Dataset
import warnings

import torch.nn as nn
import torch.nn.functional as F
import scipy.sparse as sp
from sklearn import preprocessing
import dgl
torch.set_printoptions(profile="full")
warnings.filterwarnings('ignore')

In [21]:
import torch.nn.functional as F
from sklearn.utils import shuffle


#df = pd.read_csv('training_10rep_fraclabel.csv')
df = pd.read_csv('training_5rep_fraclabel.csv')

df.drop("Unnamed: 0", axis = 1, inplace=True)
dlist = []
for i in range(1638):
    dlist.append(str(i))
    dlist.append('label')
   

#df = df.drop_duplicates(subset=dlist, keep='first', inplace=False)
#df = df.sample(n = 50000, random_state=1)
#df = shuffle(df, random_state=1)

In [22]:
df.describe()

0         1         2         3         4         5  \
count  127781.0  127781.0  127781.0  127781.0  127781.0  127781.0   
mean        0.0       0.0       0.0       0.0       0.0       0.0   
std         0.0       0.0       0.0       0.0       0.0       0.0   
min         0.0       0.0       0.0       0.0       0.0       0.0   
25%         0.0       0.0       0.0       0.0       0.0       0.0   
50%         0.0       0.0       0.0       0.0       0.0       0.0   
75%         0.0       0.0       0.0       0.0       0.0       0.0   
max         0.0       0.0       0.0       0.0       0.0       0.0   

                   6              7              8              9  \
count  127781.000000  127781.000000  127781.000000  127781.000000   
mean        0.999077       0.998427       0.997120       0.998803   
std         0.030374       0.039630       0.053588       0.034582   
min         0.000000       0.000000       0.000000       0.000000   
25%         1.000000       1.000000       1.000000       1.000000   
50%         1.000000       1.000000       1.000000       1.000000   
75%         1.000000       1.000000       1.000000       1.000000   
max         1.000000       1.000000       1.000000       1.000000   

                  10        11        12        13        14        15  \
count  127781.000000  127781.0  127781.0  127781.0  127781.0  127781.0   
mean        0.999037       0.0       0.0       0.0       0.0       0.0   
std         0.031011       0.0       0.0       0.0       0.0       0.0   
min         0.000000       0.0       0.0       0.0       0.0       0.0   
25%         1.000000       0.0       0.0       0.0       0.0       0.0   
50%         1.000000       0.0       0.0       0.0       0.0       0.0   
75%         1.000000       0.0       0.0       0.0       0.0       0.0   
max         1.000000       0.0       0.0       0.0       0.0       0.0   

             16        17        18        19        20        21        22  \
count  127781.0  127781.0  127781.0  127781.0  127781.0  127781.0  127781.0   
mean        0.0       0.0       0.0       0.0       0.0       0.0       0.0   
std         0.0       0.0       0.0       0.0       0.0       0.0       0.0   
min         0.0       0.0       0.0       0.0       0.0       0.0       0.0   
25%         0.0       0.0       0.0       0.0       0.0       0.0       0.0   
50%         0.0       0.0       0.0       0.0       0.0       0.0       0.0   
75%         0.0       0.0       0.0       0.0       0.0       0.0       0.0   
max         0.0       0.0       0.0       0.0       0.0       0.0       0.0   

             23        24        25        26             27        28  \
count  127781.0  127781.0  127781.0  127781.0  127781.000000  127781.0   
mean        0.0       0.0       0.0       0.0       0.999523       0.0   
std         0.0       0.0       0.0       0.0       0.021844       0.0   
min         0.0       0.0       0.0       0.0       0.000000       0.0   
25%         0.0       0.0       0.0       0.0       1.000000       0.0   
50%         0.0       0.0       0.0       0.0       1.000000       0.0   
75%         0.0       0.0       0.0       0.0       1.000000       0.0   
max         0.0       0.0       0.0       0.0       1.000000       0.0   

             29        30        31             32             33  \
count  127781.0  127781.0  127781.0  127781.000000  127781.000000   
mean        0.0       0.0       0.0       0.999053       0.999131   
std         0.0       0.0       0.0       0.030758       0.029461   
min         0.0       0.0       0.0       0.000000       0.000000   
25%         0.0       0.0       0.0       1.000000       1.000000   
50%         0.0       0.0       0.0       1.000000       1.000000   
75%         0.0       0.0       0.0       1.000000       1.000000   
max         0.0       0.0       0.0       1.000000       1.000000   

                  34             35        36        37        38        39  \
count  127781.000000  1

In [ ]:
droplist = []
for i in range(441, 1638):
    if df[str(i)].sum() == 0:
        droplist.append(str(i))
#df.drop(droplist, axis=1, inplace = True)
print(droplist)

In [ ]:
weight = []
ori_weight = np.array(pd.DataFrame(df["label"].value_counts()).sort_index()).reshape(-1)
print(ori_weight)

In [ ]:
np.sum([[0,1,2],[2,1,3]],axis=0)

In [3]:
df_test = pd.read_csv('test_seed30_fraclabel.csv') 


df_test.drop("Unnamed: 0", axis = 1, inplace=True)
df_test = shuffle(df_test, random_state=1)

In [4]:
def normalize_adj(adj):
    return adj


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def graphencoder(adj):
    source = []
    target = []
    for i in range(adj.shape[0]):
        for j in range(adj.shape[1]):
            if adj[i, j]:
                source.append(i)
                target.append(j)
    source = torch.from_numpy(np.array(source))
    target = torch.from_numpy(np.array(target)) 

    adj = dgl.add_self_loop(dgl.graph((source, target),  num_nodes = 21))
   
    return adj



In [23]:
class DatasetFromCSV(Dataset):
    def __init__(self, data):
 
        self.data = data.drop("label", axis = 1)
        self.labels = np.asarray(data["label"])
        
        #self.transforms = transforms
        self.train_X, self.test_X, self.train_y, self.test_y = train_test_split(self.data,
                                                   self.labels,
                                                   train_size = 0.8,
                                                   random_state = 1)
        b, s ,t, w= np.unique(self.train_y,return_counts=True,return_index=True,return_inverse=True)
        

 
    def __getitem__(self, index):
        
        single_ec_label = self.train_y[index]
        adj = self.train_X.values[index][0:441].reshape(21, 21)
        features = self.train_X.values[index][441:1638].reshape(21, 57)
        
        
        nor_adj = graphencoder(adj)
        
        #nor_adj = (adj + sp.eye(adj.shape[0]))
        nor_features = normalize(features)
        
        #nor_adj = torch.from_numpy(nor_adj)
        nor_features = torch.from_numpy(nor_features)
        nor_adj.ndata['feats'] = nor_features
    
            
        return (nor_adj, single_ec_label) 
    
    def __len__(self):
        
        return len(self.train_X.index)

In [24]:
class TestDatasetFromCSV(Dataset):
    def __init__(self, data):
 
        self.data = data.drop("label", axis = 1)
        self.labels = np.asarray(data["label"])
        
        #self.transforms = transforms
        self.train_X, self.test_X, self.train_y, self.test_y = train_test_split(self.data,
                                                   self.labels,
                                                   train_size = 0.8,
                                                   random_state = 1)
        b, s ,t, w= np.unique(self.train_y,return_counts=True,return_index=True,return_inverse=True)
        

 
    def __getitem__(self, index):
        
        single_ec_label = self.test_y[index]
        adj = self.test_X.values[index][0:441].reshape(21, 21)
        features = self.test_X.values[index][441:1638].reshape(21, 57)
    
        nor_adj = graphencoder(adj)
        
        #nor_adj = (adj + sp.eye(adj.shape[0]))
        nor_features = normalize(features)
        
        #nor_adj = torch.from_numpy(nor_adj)
        nor_features = torch.from_numpy(nor_features)
        nor_adj.ndata['feats'] = nor_features
    
            
        return (nor_adj, single_ec_label) 
    
    def __len__(self):
        
        return len(self.test_X.index)

In [25]:
def collate(samples):
    
    graphs, labels = zip(*samples)
    batched_graph = dgl.batch(graphs)
    
    batched_labels = torch.tensor(labels)
    
    return batched_graph, batched_labels

In [26]:
from torch.utils.data import DataLoader
dataloader = DataLoader(
    DatasetFromCSV(df),
    batch_size=1,
    collate_fn=collate,
    drop_last=False,
    shuffle=True)

In [27]:
len(DatasetFromCSV(df))

102224

In [28]:
from torch.utils.data import DataLoader
dataloader_t = DataLoader(
    TestDatasetFromCSV(df),
    batch_size=1,
    collate_fn=collate,
    drop_last=False,
    shuffle=True)

In [30]:
len(TestDatasetFromCSV(df))

25557

In [12]:
from dgl.nn.pytorch import edge_softmax, GATConv
class GAT(nn.Module):
    def __init__(self,
                 num_layers,
                 in_dim,
                 num_hidden,
                 num_classes,
                 heads,
                 activation,
                 feat_drop,
                 attn_drop,
                 negative_slope,
                 residual):
        super(GAT, self).__init__()
        self.num_layers = num_layers
        self.gat_layers = nn.ModuleList()
        self.activation = activation
        # input projection (no residual)
        self.gat_layers.append(GATConv(
            in_dim, num_hidden, heads[0],
            feat_drop, attn_drop, negative_slope, False, self.activation))

        # hidden layers
        for l in range(1, num_layers):
            # due to multi-head, the in_dim = num_hidden * num_heads
            self.gat_layers.append(GATConv(
                num_hidden * heads[l - 1], num_hidden, heads[l],
                feat_drop, attn_drop, negative_slope, residual, self.activation))
        # output projection
        self.gat_layers.append(GATConv(
            num_hidden * heads[-2], num_classes, heads[-1],
            feat_drop, attn_drop, negative_slope, residual, None))

        #self.bn = nn.BatchNorm1d(num_hidden)
        self.fc = nn.Linear(num_classes, 1)
#         self.do = nn.Dropout(p = 0.5)
#         self.fc2 = nn.Linear(16, 8)
#         self.fc3 = nn.Linear(8, 1)

    def forward(self, g, inputs):
        h = inputs
        for l in range(self.num_layers):
            h = self.gat_layers[l](g, h).flatten(1)
        # output projection
        h = self.gat_layers[-1](g, h).mean(1)
        #h = self.bn(h)
        with g.local_scope():
#             h = F.relu(self.do(self.fc(h)))
#             h = F.relu(self.fc2(h))
#             h = self.fc3(h)
            #h = self.fc2(h)
            g.ndata['h'] = h
            # Calculate graph representation by average readout.
            hg = dgl.mean_nodes(g, "h")
            # print(hg.size())

            return self.fc(hg)

In [17]:

heads = ([3] * 3) + [1] 
model = GAT(num_layers = 3,
          in_dim = 57,
          num_hidden = 8,
          num_classes = 6,
          heads = heads,
          activation = F.elu,
          feat_drop = 0。1,
          attn_drop = 0,
          negative_slope = 0,
          residual = False)

model.double()

SyntaxError: invalid character in identifier (<ipython-input-17-4f7ecf5c2dbe>, line 8)

In [20]:

opt = torch.optim.Adam(model.parameters(), lr = 0.00005)
running_loss = 0
test_loss = 0
total = 0
total_t = 0
epoch_num = 0

for epoch in range(50):

    i = 0
    running_loss = 0.0
    epoch_num += 1
    total = 0
    for batched_graph, labels in dataloader:

        feats = batched_graph.ndata['feats']
        opt.zero_grad()
        model.train()
        outputs = model(batched_graph, feats)
        #print(outputs, labels)
        #raise Exception("stop")
        #loss = nn.CrossEntropyLoss()(outputs, labels)
        loss = nn.MSELoss()(outputs, labels)

        running_loss += loss.item()
        total += labels.size(0)  

        loss.backward()
        opt.step()
        i += 1

        if i % 15000 == 14999:
            correct = 0
            total_t = 0
            with torch.no_grad():
                j = 0
                for batched_graph_t, labels_t in dataloader_t:

                    feats_t = batched_graph_t.ndata['feats']
                    model.eval()
                    outputs_t = model(batched_graph_t, feats_t)

                    #loss_t = nn.CrossEntropyLoss()(outputs_t, labels_t)
                    loss_t = nn.MSELoss()(outputs_t, labels_t)
                    test_loss += loss_t.item()

                    #total_t += labels_t.size(0)  
                    #_, predicted = torch.max(outputs_t.data, 1)
                    #correct += (predicted == labels_t).sum().item()
                    #print(predicted, labels_t)
                    j += 1

            print('epoch %d' % epoch_num, 'training_loss: %.4f' % (running_loss / 10000), 'test_loss: %.4f' %(test_loss /j)) #'Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total_t))           
            running_loss = 0.0
            test_loss = 0
            total = 0



epoch 1 training_loss: 0.1617 test_loss: 0.1643
epoch 1 training_loss: 0.1639 test_loss: 0.1679
epoch 1 training_loss: 0.1659 test_loss: 0.1688
epoch 2 training_loss: 0.1613 test_loss: 0.1669
epoch 2 training_loss: 0.1606 test_loss: 0.1633
epoch 2 training_loss: 0.1620 test_loss: 0.1717
epoch 3 training_loss: 0.1576 test_loss: 0.1606
epoch 3 training_loss: 0.1601 test_loss: 0.1763
epoch 3 training_loss: 0.1616 test_loss: 0.1632
epoch 4 training_loss: 0.1576 test_loss: 0.1703
epoch 4 training_loss: 0.1594 test_loss: 0.1645
epoch 4 training_loss: 0.1569 test_loss: 0.1747
epoch 5 training_loss: 0.1549 test_loss: 0.1676
epoch 5 training_loss: 0.1589 test_loss: 0.1626
epoch 5 training_loss: 0.1579 test_loss: 0.1698
epoch 6 training_loss: 0.1558 test_loss: 0.1695
epoch 6 training_loss: 0.1573 test_loss: 0.1618
epoch 6 training_loss: 0.1566 test_loss: 0.1730
epoch 7 training_loss: 0.1539 test_loss: 0.1691
epoch 7 training_loss: 0.1534 test_loss: 0.1740
epoch 7 training_loss: 0.1599 test_loss:

KeyboardInterrupt: 

In [ ]:
torch.save({
       
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': opt.state_dict(),
      
       
        }, "gat_oct18_test.pt")

print("success")

In [ ]:
checkpoint = torch.load('gat_oct4_test.pt')
heads = ([1] * 1) + [1]
model = new_GAT(num_layers=1,
            in_dim=57,
            num_hidden=20,
            num_classes=1,
            heads=heads,
            activation=F.elu,
            feat_drop=0.1,
            attn_drop=0.3,
            negative_slope=0,
            residual=False)

model.double()
model.eval()
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
j = 0
correct = 0
total_t = 0
test_loss = 0
for batched_graph_t, labels_t in dataloader_t:
    
    feats_t = batched_graph_t.ndata['feats']
    model.eval()
    outputs_t = model(batched_graph_t, feats_t)

    loss_t = nn.CrossEntropyLoss()(outputs_t, labels_t)
    test_loss += loss_t.item()

    total_t += labels_t.size(0)  
    _, predicted = torch.max(outputs_t.data, 1)
    correct += (predicted == labels_t).sum().item()
    # print(predicted, labels_t)
    j += 1

print('test_loss: %.4f' %(test_loss /j), 'Accuracy of the network on the 10000 test images: %d %%' % (
100 * correct / total_t))

In [ ]:
0.12 loss on test data

heads = ([3] * 3) + [1] 
model = GAT(num_layers = 3,
          in_dim = 57,
          num_hidden = 8,
          num_classes = 6,
          heads = heads,
          activation = F.elu,
          feat_drop = 0,
          attn_drop = 0,
          negative_slope = 0,
          residual = False)

model.double()

In [ ]:
epoch 1 training_loss: 0.2334 test_loss: 0.1753
epoch 1 training_loss: 0.2300 test_loss: 0.1742
epoch 1 training_loss: 0.2262 test_loss: 0.1677
epoch 2 training_loss: 0.2244 test_loss: 0.1630
epoch 2 training_loss: 0.2219 test_loss: 0.1648
epoch 2 training_loss: 0.2199 test_loss: 0.1633
epoch 3 training_loss: 0.2167 test_loss: 0.1593
epoch 3 training_loss: 0.2137 test_loss: 0.1556
epoch 3 training_loss: 0.2145 test_loss: 0.1547
epoch 4 training_loss: 0.2125 test_loss: 0.1557
epoch 4 training_loss: 0.2061 test_loss: 0.1531
epoch 4 training_loss: 0.2114 test_loss: 0.1563
epoch 5 training_loss: 0.2075 test_loss: 0.1484
epoch 5 training_loss: 0.2073 test_loss: 0.1495
epoch 5 training_loss: 0.2034 test_loss: 0.1566
epoch 6 training_loss: 0.2031 test_loss: 0.1532
epoch 6 training_loss: 0.2032 test_loss: 0.1576
epoch 6 training_loss: 0.2048 test_loss: 0.1547
epoch 7 training_loss: 0.2019 test_loss: 0.1558
epoch 7 training_loss: 0.1982 test_loss: 0.1538
epoch 7 training_loss: 0.2033 test_loss: 0.1574
epoch 8 training_loss: 0.2004 test_loss: 0.1605
epoch 8 training_loss: 0.1991 test_loss: 0.1542
epoch 8 training_loss: 0.1989 test_loss: 0.1537
epoch 9 training_loss: 0.2010 test_loss: 0.1655
epoch 9 training_loss: 0.1992 test_loss: 0.1640
epoch 9 training_loss: 0.1949 test_loss: 0.1668
epoch 10 training_loss: 0.1920 test_loss: 0.1747
epoch 10 training_loss: 0.1985 test_loss: 0.1880
epoch 10 training_loss: 0.1977 test_loss: 0.1598
epoch 11 training_loss: 0.1958 test_loss: 0.1736
epoch 11 training_loss: 0.1966 test_loss: 0.1865
epoch 11 training_loss: 0.1959 test_loss: 0.1797
epoch 12 training_loss: 0.1951 test_loss: 0.1875
epoch 12 training_loss: 0.1941 test_loss: 0.1916
epoch 12 training_loss: 0.1958 test_loss: 0.1590
epoch 13 training_loss: 0.1899 test_loss: 0.1695
epoch 13 training_loss: 0.1941 test_loss: 0.1808
epoch 13 training_loss: 0.1948 test_loss: 0.1585
epoch 14 training_loss: 0.1889 test_loss: 0.1753
epoch 14 training_loss: 0.1944 test_loss: 0.1657
epoch 14 training_loss: 0.1925 test_loss: 0.1542
epoch 15 training_loss: 0.1906 test_loss: 0.1515
epoch 15 training_loss: 0.1890 test_loss: 0.1597
epoch 15 training_loss: 0.1907 test_loss: 0.1641
epoch 16 training_loss: 0.1936 test_loss: 0.1494
epoch 16 training_loss: 0.1875 test_loss: 0.1575
epoch 16 training_loss: 0.1885 test_loss: 0.1696
epoch 17 training_loss: 0.1871 test_loss: 0.1579
epoch 17 training_loss: 0.1885 test_loss: 0.1573
epoch 17 training_loss: 0.1885 test_loss: 0.1524
epoch 18 training_loss: 0.1928 test_loss: 0.1550
epoch 18 training_loss: 0.1849 test_loss: 0.1633
epoch 18 training_loss: 0.1885 test_loss: 0.1567
epoch 19 training_loss: 0.1871 test_loss: 0.1597
epoch 19 training_loss: 0.1840 test_loss: 0.1557
epoch 19 training_loss: 0.1879 test_loss: 0.1495
epoch 20 training_loss: 0.1861 test_loss: 0.1593
epoch 20 training_loss: 0.1876 test_loss: 0.1575
epoch 20 training_loss: 0.1845 test_loss: 0.1522
epoch 21 training_loss: 0.1848 test_loss: 0.1638
epoch 21 training_loss: 0.1851 test_loss: 0.1602
epoch 21 training_loss: 0.1864 test_loss: 0.1606
epoch 22 training_loss: 0.1849 test_loss: 0.1602
epoch 22 training_loss: 0.1833 test_loss: 0.1571
epoch 22 training_loss: 0.1855 test_loss: 0.1659
epoch 23 training_loss: 0.1842 test_loss: 0.2020
epoch 23 training_loss: 0.1840 test_loss: 0.1834
epoch 23 training_loss: 0.1839 test_loss: 0.1740
epoch 24 training_loss: 0.1803 test_loss: 0.1616
epoch 24 training_loss: 0.1853 test_loss: 0.1648
epoch 24 training_loss: 0.1851 test_loss: 0.1582
epoch 25 training_loss: 0.1853 test_loss: 0.1951
epoch 25 training_loss: 0.1841 test_loss: 0.1601
epoch 25 training_loss: 0.1814 test_loss: 0.1620
epoch 26 training_loss: 0.1820 test_loss: 0.1655
epoch 26 training_loss: 0.1828 test_loss: 0.1758
epoch 26 training_loss: 0.1832 test_loss: 0.1629
epoch 27 training_loss: 0.1807 test_loss: 0.1661
epoch 27 training_loss: 0.1806 test_loss: 0.1557
epoch 27 training_loss: 0.1840 test_loss: 0.1744
epoch 28 training_loss: 0.1809 test_loss: 0.1698
epoch 28 training_loss: 0.1848 test_loss: 0.1773
epoch 28 training_loss: 0.1814 test_loss: 0.1727
epoch 29 training_loss: 0.1786 test_loss: 0.1831
epoch 29 training_loss: 0.1814 test_loss: 0.1741
epoch 29 training_loss: 0.1787 test_loss: 0.1910
epoch 30 training_loss: 0.1781 test_loss: 0.2091
epoch 30 training_loss: 0.1824 test_loss: 0.1842
epoch 30 training_loss: 0.1829 test_loss: 0.1815
epoch 31 training_loss: 0.1780 test_loss: 0.1699
epoch 31 training_loss: 0.1786 test_loss: 0.1965
epoch 31 training_loss: 0.1835 test_loss: 0.1836
epoch 32 training_loss: 0.1781 test_loss: 0.2027
epoch 32 training_loss: 0.1811 test_loss: 0.2014
epoch 32 training_loss: 0.1783 test_loss: 0.1717
epoch 33 training_loss: 0.1795 test_loss: 0.1927
epoch 33 training_loss: 0.1760 test_loss: 0.1721
epoch 33 training_loss: 0.1812 test_loss: 0.1744
epoch 34 training_loss: 0.1789 test_loss: 0.1840
epoch 34 training_loss: 0.1819 test_loss: 0.1878
epoch 34 training_loss: 0.1782 test_loss: 0.1680
epoch 35 training_loss: 0.1787 test_loss: 0.1857
epoch 35 training_loss: 0.1791 test_loss: 0.1759
epoch 35 training_loss: 0.1760 test_loss: 0.1579
epoch 36 training_loss: 0.1766 test_loss: 0.1660
epoch 36 training_loss: 0.1771 test_loss: 0.1851
epoch 36 training_loss: 0.1766 test_loss: 0.1905
epoch 37 training_loss: 0.1773 test_loss: 0.1706
epoch 37 training_loss: 0.1757 test_loss: 0.1793
epoch 37 training_loss: 0.1783 test_loss: 0.1777
epoch 38 training_loss: 0.1788 test_loss: 0.1789
epoch 38 training_loss: 0.1787 test_loss: 0.1797
epoch 38 training_loss: 0.1773 test_loss: 0.1586
epoch 39 training_loss: 0.1751 test_loss: 0.1646
epoch 39 training_loss: 0.1786 test_loss: 0.1645
epoch 39 training_loss: 0.1760 test_loss: 0.1543
epoch 40 training_loss: 0.1755 test_loss: 0.1539
epoch 40 training_loss: 0.1758 test_loss: 0.1522
epoch 40 training_loss: 0.1750 test_loss: 0.1870
epoch 41 training_loss: 0.1780 test_loss: 0.1862
epoch 41 training_loss: 0.1729 test_loss: 0.1616
epoch 41 training_loss: 0.1746 test_loss: 0.1455
epoch 42 training_loss: 0.1726 test_loss: 0.1821
epoch 42 training_loss: 0.1763 test_loss: 0.1533
epoch 42 training_loss: 0.1758 test_loss: 0.1627
epoch 43 training_loss: 0.1736 test_loss: 0.1626
epoch 43 training_loss: 0.1733 test_loss: 0.1586
epoch 43 training_loss: 0.1756 test_loss: 0.1784
epoch 44 training_loss: 0.1746 test_loss: 0.1760
epoch 44 training_loss: 0.1759 test_loss: 0.1686
epoch 44 training_loss: 0.1739 test_loss: 0.1704
epoch 45 training_loss: 0.1737 test_loss: 0.1642
epoch 45 training_loss: 0.1749 test_loss: 0.1834
epoch 45 training_loss: 0.1741 test_loss: 0.1797
epoch 46 training_loss: 0.1730 test_loss: 0.1817
epoch 46 training_loss: 0.1724 test_loss: 0.1612
epoch 46 training_loss: 0.1749 test_loss: 0.1776
